# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("rec").getOrCreate()

22/10/04 21:28:53 WARN Utils: Your hostname, masoud-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.7.139 instead (on interface wlp2s0)
22/10/04 21:28:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 21:28:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/04 21:28:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(
    maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating"
)
model = als.fit(training)

Now let's see hwo the model performed!

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      5|   2.0|    26|   2.225257|
|      6|   3.0|    26|   1.743498|
|      2|   1.0|    12|  0.6892269|
|      6|   1.0|    12|  1.4339107|
|      3|   1.0|     1| 0.95839715|
|      0|   1.0|    13|  1.1116412|
|      5|   1.0|    13| 0.63104147|
|      1|   1.0|     6| 0.73474365|
|      5|   1.0|     6| -0.4268195|
|      6|   1.0|     6|  0.9563544|
|      6|   2.0|    16|0.048763394|
|      2|   2.0|    20|   3.148292|
|      0|   1.0|     5|  0.9526024|
|      2|   1.0|    19| 0.44643763|
|      2|   1.0|    15| -2.7412846|
|      4|   2.0|     8|-0.16986287|
|      3|   1.0|    29| 0.79041433|
|      3|   1.0|    21|  2.3618803|
|      5|   1.0|    14| 0.94014037|
|      1|   1.0|    18|  2.2778506|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.70377060341621


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [12]:
single_user = test.filter(test["userId"] == 11).select(["movieId", "userId"])

In [13]:
# User had 10 ratings in the test data set
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     16|    11|
|     32|    11|
|     47|    11|
|     51|    11|
|     76|    11|
|     82|    11|
|     99|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy("prediction", ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     16|    11| 3.5192776|
|     76|    11|  2.856176|
|     99|    11| 2.1528373|
|     51|    11| 1.8586149|
|     82|    11|  1.422145|
|     32|    11|0.81392634|
|     47|    11| 0.7377104|
+-------+------+----------+



# Great Job!